In [1]:
%config Completer.use_jedi = False

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.datasets import cifar10

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [4]:
(train_X, train_y), (testData, testLabels) = cifar10.load_data()

In [5]:
CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [9]:
print(train_X.shape)
print(train_y.shape)
print(testData.shape)
print(testLabels.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [8]:
trainData, valData, trainLabels, valLabels = train_test_split(train_X, train_y, test_size=0.10, random_state=42)

In [12]:
print(trainData.shape)
print(trainLabels.shape)
print(valData.shape)
print(valLabels.shape)

(45000, 32, 32, 3)
(45000, 1)
(5000, 32, 32, 3)
(5000, 1)


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))